In [1]:
from sklearn import datasets
import scipy.io as scio
import numpy as np
traindata = scio.loadmat('E:\\.000心电\\实验室友情赞助文件\\彩云\\AF_fea_data\\10s\\traindataz.mat')#traindata = scio.loadmat('E:\\00心电\\af\\traindata.mat')#E:\00心电\af
data = traindata['traindataz']
trainlable = scio.loadmat('E:\\.000心电\\实验室友情赞助文件\\彩云\\AF_fea_data\\10s\\trainlable')#trainlable = scio.loadmat('E:\\00心电\\af\\trainlable.mat') 
lable = trainlable['trainlable'][0]
#traindata = scio.loadmat('E:\\00心电\\af\\traindata.mat')
import pandas as pd

In [2]:
testdata = scio.loadmat('E:\\.000心电\\实验室友情赞助文件\\彩云\\AF_fea_data\\10s\\2018挑战赛10s\\testdata201810s.mat')#traindata = scio.loadmat('E:\\00心电\\af\\traindata.mat')#E:\00心电\af
testdata = testdata['testdata201810s'][:,0:7]
testlable = scio.loadmat('E:\\.000心电\\实验室友情赞助文件\\彩云\\AF_fea_data\\10s\\2018挑战赛10s\\testtable201810s.mat')#trainlable = scio.loadmat('E:\\00心电\\af\\trainlable.mat') 
testlable = testlable['testtable201810s'][:,0]

In [3]:
np.random.seed(1)
np.random.shuffle(data) 
np.random.seed(1)
np.random.shuffle(lable)
np.random.seed(1)
np.random.shuffle(testdata) 
np.random.seed(1)
np.random.shuffle(testlable)

import torch
input = torch.FloatTensor(data)
label = torch.LongTensor(lable)
input_test = torch.FloatTensor(testdata)
label_test = torch.LongTensor(testlable)

input_train = input[0:60001]
label_train = label[0:60001]

input_valid = input[60001:]
label_valid = label[60001:]

In [4]:
import torch.nn.functional as Fun
# 定义BP神经网络
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)
        self.out = torch.nn.Linear(n_hidden, n_output)
        
    def forward(self,x):
        x = Fun.relu(self.hidden(x))
        x = self.out(x)
        return x

In [136]:
Lay = [7,1024,2]

In [137]:
#思路是训练，然后脚本
net = Net(n_feature=Lay[0], n_hidden=Lay[1],n_output=Lay[2])
#optimizer = torch.optim.SGD(net.parameters(), lr=0.05)
optimizer = torch.optim.Adam(net.parameters())
# SGD:随机梯度下降法
loss_func = torch.nn.CrossEntropyLoss()
# 设定损失函数


#开始训练
#并在每100轮输出结果（这里最好每轮都记录我们的损失和精度，需要改进代码）
losslist = []
speficlist = []
spefictestlist = []
spefictrainlist = []
for i in range(200):
    out = net(input_train)
    loss = loss_func(out, label_train)
    if(i%1 == 0):
        #print(loss)
        losslist.append(loss)
        out_test = net(input_valid)
        # out是一个计算矩阵
        prediction = torch.max(out_test, 1)[1]
        pred_y = prediction.numpy()
        # 预测y输出数列
        target_y = label_valid.data.numpy()
        speficvalid = np.sum(pred_y == target_y)/(np.sum(pred_y == target_y) +np.sum(pred_y != target_y) )
        speficlist.append(speficvalid)
        #plt.figure('Draw')

        out_train = net(input_train)
        # out是一个计算矩阵
        prediction = torch.max(out_train, 1)[1]
        pred_y = prediction.numpy()
        # 预测y输出数列
        target_y = label_train.data.numpy()
        spefictrain = np.sum(pred_y == target_y)/(np.sum(pred_y == target_y) +np.sum(pred_y != target_y) )
        spefictrainlist.append(spefictrain)
        
        out_test = net(input_test)
        # out是一个计算矩阵
        prediction = torch.max(out_test, 1)[1]
        pred_y = prediction.numpy()
        # 预测y输出数列
        target_y = label_test.data.numpy()
        spefic = np.sum(pred_y == target_y)/(np.sum(pred_y == target_y) +np.sum(pred_y != target_y) )
        spefictestlist.append(spefic)
        
        print(i,loss,spefictrain,speficvalid,spefic)
        #plt.plot(speficlist)  # plot绘制折线图
        #plt.draw()  # 显示绘图
        #plt.pause(2)  #显示5秒
        #plt.close()
    # 输出与label对比
#     if(i > 150 and i%10 == 0):
#         torch.save(net.state_dict(), '3_layer7-8-2_'+str(i)+'.pt')
    optimizer.zero_grad()
    # 初始化
    loss.backward()
    optimizer.step()

0 tensor(0.7828, grad_fn=<NllLossBackward>) 0.49704171597140046 0.4987679373822293 0.5
1 tensor(0.6800, grad_fn=<NllLossBackward>) 0.5000416659722338 0.49963762864183214 0.6080931263858093
2 tensor(0.6092, grad_fn=<NllLossBackward>) 0.5093415109748171 0.5064502101753877 0.8292682926829268
3 tensor(0.5302, grad_fn=<NllLossBackward>) 0.844085931901135 0.8411363965792144 0.9478935698447893
4 tensor(0.4656, grad_fn=<NllLossBackward>) 0.9247679205346577 0.9202783012030729 0.9279379157427938
5 tensor(0.4195, grad_fn=<NllLossBackward>) 0.924551257479042 0.9228873749818814 0.8957871396895787
6 tensor(0.3809, grad_fn=<NllLossBackward>) 0.9221846302561624 0.92245252935208 0.9029933481152993
7 tensor(0.3444, grad_fn=<NllLossBackward>) 0.9259345677572041 0.9246267575010871 0.9279379157427938
8 tensor(0.3129, grad_fn=<NllLossBackward>) 0.9293511774803753 0.9273807798231628 0.9523281596452328
9 tensor(0.2891, grad_fn=<NllLossBackward>) 0.9288845185913568 0.9249166545876214 0.9628603104212861
10 tens

81 tensor(0.1756, grad_fn=<NllLossBackward>) 0.9366843885935234 0.9359327438759241 0.9611973392461197
82 tensor(0.1754, grad_fn=<NllLossBackward>) 0.9369343844269262 0.9362226409624583 0.9634146341463414
83 tensor(0.1751, grad_fn=<NllLossBackward>) 0.9370510491491808 0.9363675895057255 0.9656319290465631
84 tensor(0.1748, grad_fn=<NllLossBackward>) 0.9371177147047549 0.9365125380489926 0.9667405764966741
85 tensor(0.1745, grad_fn=<NllLossBackward>) 0.9372343794270096 0.9368024351355269 0.9667405764966741
86 tensor(0.1743, grad_fn=<NllLossBackward>) 0.937251045815903 0.9366574865922598 0.9684035476718403
87 tensor(0.1740, grad_fn=<NllLossBackward>) 0.9373343777603707 0.9368024351355269 0.9684035476718403
88 tensor(0.1737, grad_fn=<NllLossBackward>) 0.9374677088715188 0.936947383678794 0.9684035476718403
89 tensor(0.1735, grad_fn=<NllLossBackward>) 0.9375843735937734 0.936947383678794 0.9700665188470067
90 tensor(0.1732, grad_fn=<NllLossBackward>) 0.9378343694271762 0.9370923322220611 0.

162 tensor(0.1581, grad_fn=<NllLossBackward>) 0.944450925817903 0.9447746050152196 0.9739467849223947
163 tensor(0.1579, grad_fn=<NllLossBackward>) 0.9445509241512642 0.9447746050152196 0.9733924611973392
164 tensor(0.1577, grad_fn=<NllLossBackward>) 0.9446509224846252 0.9450645021017539 0.9733924611973392
165 tensor(0.1576, grad_fn=<NllLossBackward>) 0.9447509208179864 0.9450645021017539 0.9733924611973392
166 tensor(0.1574, grad_fn=<NllLossBackward>) 0.9447509208179864 0.945209450645021 0.9733924611973392
167 tensor(0.1572, grad_fn=<NllLossBackward>) 0.9448175863735604 0.9454993477315553 0.9733924611973392
168 tensor(0.1571, grad_fn=<NllLossBackward>) 0.9449675838736021 0.9456442962748225 0.9711751662971175
169 tensor(0.1569, grad_fn=<NllLossBackward>) 0.9451175813736438 0.9457892448180896 0.970620842572062
170 tensor(0.1567, grad_fn=<NllLossBackward>) 0.9451009149847502 0.9459341933613568 0.9711751662971175
171 tensor(0.1566, grad_fn=<NllLossBackward>) 0.9451009149847502 0.946079141

In [138]:
cpp_front= open("AF_classifier_front_7_x_2.cpp")  
#逐行写入
for line in cpp_front.readlines():  
    with open("AF_classifier_"+str(Lay[0])+"_"+str(Lay[1])+"_"+str(Lay[2])+".cpp","a") as cpp:
        cpp.write(line)  
cpp_front.close()

In [139]:
#将模型参数转换为C/C++中的数组写入到文本中
fp = open("AF_classifier_"+str(Lay[0])+"_"+str(Lay[1])+"_"+str(Lay[2])+".cpp",'a')
flag = True
k=1
for name,param in net.named_parameters():  

    print(param)
    temp =  param
    temp = temp.float()
    print(temp)
    temp1 = temp.numpy
    temp2= temp.tolist()
    if flag == True:
        flag = False
        fp.write('ap_f Hw'+str(k)+'['+str(len(temp2))+']['+str(len(temp2[0]))+'] = {')
        for i in range(len(temp2)-1):
            fp.write('{')
            for j in range(len(temp2[0])-1):
                fp.write(str(temp2[i][j])+',')
            fp.write(str(temp2[i][len(temp2[0])-1])+'},\n')
        #尾巴
        fp.write('{')
        for j in range(len(temp2[0])-1):
            fp.write(str(temp2[len(temp2)-1][j])+',')
        fp.write(str(temp2[len(temp2)-1][len(temp2[0])-1])+'}')
        fp.write('};\n\n\n')
    else:
        flag = True
        fp.write('ap_f Hb'+str(k)+'['+str(len(temp2))+'] = {')
        k +=1
        for i in range(len(temp2)-1):
            fp.write(str(temp2[i])+',')
        fp.write(str(temp2[len(temp2)-1]))
        fp.write('};\n\n\n')
        
fp.close()

Parameter containing:
tensor([[-0.0651,  0.3904,  0.1375,  ...,  0.2665, -0.0235, -0.3006],
        [ 0.3089,  0.1239,  0.1991,  ..., -0.0222,  0.0841,  0.1126],
        [-0.2340,  0.0752,  0.3128,  ..., -0.1739, -0.0416, -0.1253],
        ...,
        [ 0.1609,  0.1990, -0.2910,  ..., -0.1702,  0.4801, -0.0928],
        [-0.2084, -0.0612, -0.3301,  ...,  0.2490,  0.0828, -0.3014],
        [ 0.3197,  0.1642,  0.3659,  ..., -0.2284, -0.1556,  0.0070]],
       requires_grad=True)
Parameter containing:
tensor([[-0.0651,  0.3904,  0.1375,  ...,  0.2665, -0.0235, -0.3006],
        [ 0.3089,  0.1239,  0.1991,  ..., -0.0222,  0.0841,  0.1126],
        [-0.2340,  0.0752,  0.3128,  ..., -0.1739, -0.0416, -0.1253],
        ...,
        [ 0.1609,  0.1990, -0.2910,  ..., -0.1702,  0.4801, -0.0928],
        [-0.2084, -0.0612, -0.3301,  ...,  0.2490,  0.0828, -0.3014],
        [ 0.3197,  0.1642,  0.3659,  ..., -0.2284, -0.1556,  0.0070]],
       requires_grad=True)
Parameter containing:
tensor([ 0.3

In [140]:
#将参数对应的程序转换为C/C++中的数组写入到文本中

fp = open("AF_classifier_"+str(Lay[0])+"_"+str(Lay[1])+"_"+str(Lay[2])+".cpp",'a')
for i in range(1,len(Lay)):
    print("ap_f "+chr(ord('A')+i)+'['+str(Lay[i])+'];')
    fp.write("ap_f "+chr(ord('A')+i)+'['+str(Lay[i])+'];\n')
fp.close()

ap_f B[1024];
ap_f C[2];


In [141]:
fp = open("AF_classifier_"+str(Lay[0])+"_"+str(Lay[1])+"_"+str(Lay[2])+".cpp",'a')
for i in range(len(Lay)-2):
    print('    for (int i = 0; i < '+str(Lay[i+1])+'; i++) {')
    print('        '+chr(ord('A')+i+1)+'[i] = 0;')
    print('        for (int j = 0; j < '+str(Lay[i])+'; j++) {')
    print('            '+chr(ord('A')+i+1)+'[i] = '+chr(ord('A')+i+1)+'[i] + '+chr(ord('A')+i)+'[j] * Hw'+str(i+1)+'[i][j];')
    print('        }')
    print('        '+chr(ord('A')+i+1)+'[i] = '+chr(ord('A')+i+1)+'[i]/int_para + Hb'+str(i+1)+'[i] ;')
    print('        if('+chr(ord('A')+i+1)+'[i] < 0){'+chr(ord('A')+i+1)+'[i] = 0;}')
    print('    }')
    
    fp.write('    for (int i = 0; i < '+str(Lay[i+1])+'; i++) {\n')
    fp.write('        '+chr(ord('A')+i+1)+'[i] = 0;\n')
    fp.write('        for (int j = 0; j < '+str(Lay[i])+'; j++) {\n')
    fp.write('            '+chr(ord('A')+i+1)+'[i] = '+chr(ord('A')+i+1)+'[i] + '+chr(ord('A')+i)+'[j] * Hw'+str(i+1)+'[i][j];\n')
    fp.write('        }\n')
    fp.write('        '+chr(ord('A')+i+1)+'[i] = '+chr(ord('A')+i+1)+'[i]/int_para + Hb'+str(i+1)+'[i] ;\n')
    fp.write('        if('+chr(ord('A')+i+1)+'[i] < 0){'+chr(ord('A')+i+1)+'[i] = 0;}\n')
    fp.write('    }\n')   
fp.close()

    for (int i = 0; i < 1024; i++) {
        B[i] = 0;
        for (int j = 0; j < 7; j++) {
            B[i] = B[i] + A[j] * Hw1[i][j];
        }
        B[i] = B[i]/int_para + Hb1[i] ;
        if(B[i] < 0){B[i] = 0;}
    }


In [142]:
fp = open("AF_classifier_"+str(Lay[0])+"_"+str(Lay[1])+"_"+str(Lay[2])+".cpp",'a')
for i in range(len(Lay)-2,len(Lay)-1):
    print('    for (int i = 0; i < '+str(Lay[i+1])+'; i++) {')
    print('        '+chr(ord('A')+i+1)+'[i] = 0;')
    print('        for (int j = 0; j < '+str(Lay[i])+'; j++) {')
    print('            '+chr(ord('A')+i+1)+'[i] = '+chr(ord('A')+i+1)+'[i] + '+chr(ord('A')+i)+'[j] * Hw'+str(i+1)+'[i][j];')
    print('        }')
    print('        '+chr(ord('A')+i+1)+'[i] = '+chr(ord('A')+i+1)+'[i]/int_para + Hb'+str(i+1)+'[i] ;')
    print('    }')
    #开始写入result
    print('    if ('+chr(ord('A')+i+1)+'[0] > '+chr(ord('A')+i+1)+'[1])')
    print('        {result = 0 ;}')
    print('    else')
    print('        {result = 1 ;}')
    print('}')
    
    fp.write('    for (int i = 0; i < '+str(Lay[i+1])+'; i++) {\n')
    fp.write('        '+chr(ord('A')+i+1)+'[i] = 0;\n')
    fp.write('        for (int j = 0; j < '+str(Lay[i])+'; j++) {\n')
    fp.write('            '+chr(ord('A')+i+1)+'[i] = '+chr(ord('A')+i+1)+'[i] + '+chr(ord('A')+i)+'[j] * Hw'+str(i+1)+'[i][j];\n')
    fp.write('        }\n')
    fp.write('        '+chr(ord('A')+i+1)+'[i] = '+chr(ord('A')+i+1)+'[i]/int_para + Hb'+str(i+1)+'[i] ;\n')
    fp.write('    }\n')
    #开始写入result
    fp.write('    if ('+chr(ord('A')+i+1)+'[0] > '+chr(ord('A')+i+1)+'[1])\n')
    fp.write('        {result = 0 ;}\n')
    fp.write('    else\n')
    fp.write('        {result = 1 ;}\n')
    fp.write('}\n')
fp.close()

    for (int i = 0; i < 2; i++) {
        C[i] = 0;
        for (int j = 0; j < 1024; j++) {
            C[i] = C[i] + B[j] * Hw2[i][j];
        }
        C[i] = C[i]/int_para + Hb2[i] ;
    }
    if (C[0] > C[1])
        {result = 0 ;}
    else
        {result = 1 ;}
}


In [143]:
####写入tcl
#写入tcl
with open("script.tcl","a") as tcl:
    tcl.write("\n\n\n########################\nopen_project HLS_"+str(Lay[0])+"_"+str(Lay[1])+"_"+str(Lay[2])+".cpp"+"\n")
    tcl.write("set_top AF_classifier\n")
    tcl.write("add_files "+"AF_classifier_"+str(Lay[0])+"_"+str(Lay[1])+"_"+str(Lay[2])+".cpp\n")

tcl_end = open("script_end.tcl")  
for line in tcl_end.readlines():  
    with open("script.tcl","a") as tcl:
        tcl.write(line)  
tcl_end.close()    